<a href="https://colab.research.google.com/github/DevanMayer/EE-480-Assignments/blob/main/ca4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EE 480 - Coding Assignment #4: Building the Ybus Matrix**

**Instructor:** Dr. Saeed Manshadi

**Due Date:** September 30, 2025

### **Instructions**

This assignment focuses on the fundamental skill of constructing the bus admittance matrix (Ybus), which is the foundation for all power flow studies. You will apply the "rules of inspection" to build the Ybus for two different power systems.


**Submission:**
*   Please follow the submission instructions provided on Canvas to save your completed notebook back to your private GitHub repository.


---
### **Problem 1**

Determine the bus admittance matrix **Ybus** for the five-bus power system shown in the Figure. Use the input data given in Table 1. The partially completed Ybus below is provided to help you check your work. Assume a three-phase 100 MVA per unit base.

<img src="https://i.imgur.com/bVPNYkH.png" width=400>


**Guidance:**
*   This is a direct application of building the Ybus "by inspection".
*   The data in Table 1 gives you the **series impedance** (`z = R + jX`) and the **total shunt susceptance** (`B`) for each line. The total shunt admittance is `y_sh = jB`. Remember that the shunt admittance is split in half at each end of the line in the π-model.
*   **Step 1:** For each line, calculate its series admittance `y_series = 1 / z_series`.
*   **Step 2:** Initialize a 5x5 matrix of complex zeros for your Ybus.
*   **Step 3:** Populate the **off-diagonal** elements using the rule `Y_ij = -y_series_ij`.
*   **Step 4:** Populate the **diagonal** elements using the rule `Y_ii = Σ y_series_ik + Σ y_shunt_i`. The total shunt admittance at a bus is the sum of the `B/2` contributions from every line connected to it.

In [2]:
# --- Problem 1 Solution ---
using LinearAlgebra

# --- Given Data from Table 1 ---
# Branch series impedances (R + jX)
z12 = 0.02 + 0.06*im
z13 = 0.08 + 0.24*im
z23 = 0.06 + 0.18*im
z24 = 0.08 + 0.24*im
z25 = 0.02 + 0.06*im
z34 = 0.01 + 0.04*im
z45 = 0.03 + 0.10*im

# Branch total shunt susceptances (B). Admittance is y_sh = jB
B12 = 0.06
B13 = 0.05
B23 = 0.04
B24 = 0.05
B25 = 0.02
B34 = 0.01
B45 = 0.04

# --- Step 1: Calculate Series Admittances ---
y12 = 1/z12
y13 = 1/z13
y23 = 1/z23
y24 = 1/z24
y25 = 1/z25
y34 = 1/z34
y45 = 1/z45

# --- Step 2: Initialize Ybus ---
N = 5
Ybus = zeros(Complex{Float64}, N, N)

# --- Step 3: Populate Off-Diagonal Elements ---
Ybus[1, 2] = Ybus[2, 1] = -y12
Ybus[1, 3] = Ybus[3, 1] = -y13
Ybus[2, 3] = Ybus[3, 2] = -y23
Ybus[2, 4] = Ybus[4, 2] = -y24
Ybus[2, 5] = Ybus[5, 2] = -y25
Ybus[3, 4] = Ybus[4, 3] = -y34
Ybus[4, 5] = Ybus[5, 4] = -y45

# --- Step 4: Populate Diagonal Elements ---
# Y_ii = (sum of series admittances connected to bus i) + (sum of shunt admittances at bus i)
# Remember that the shunt B is split, so each bus gets B/2 from a connected line.
Ybus[1, 1] = y12 + y13 + im*B12/2 + im*B13/2
Ybus[2, 2] = y12 + y23 + y24 + y25 + im*(B12/2 + B23/2 + B24/2 + B25/2)
Ybus[3, 3] = y13+y23+y34+im*(B13/2 + B23/2 + B34/2)
Ybus[4, 4] = y24+y34+y45+im*(B24/2 +B34/2 +B45/2)
Ybus[5, 5] = y25+y45+im*(B25/2 +B45/2)

println("--- Final Ybus Matrix for Problem 1 ---")
# Use the show function to display the full matrix without truncation
show(IOContext(stdout, :limit => false), "text/plain", round.(Ybus, digits=4))

--- Final Ybus Matrix for Problem 1 ---
5×5 Matrix{ComplexF64}:
  6.25-18.695im     -5.0+15.0im      -1.25+3.75im         0.0+0.0im          0.0+0.0im
  -5.0+15.0im    12.9167-38.665im  -1.6667+5.0im        -1.25+3.75im        -5.0+15.0im
 -1.25+3.75im    -1.6667+5.0im       8.799-32.2294im  -5.8824+23.5294im      0.0+0.0im
   0.0+0.0im       -1.25+3.75im    -5.8824+23.5294im   9.8846-36.4037im  -2.7523+9.1743im
   0.0+0.0im        -5.0+15.0im        0.0+0.0im      -2.7523+9.1743im    7.7523-24.1443im


---
### **Problem 2**

For the three-bus system shown below, the bus admittances are given in per-unit. Determine the type of each bus and edit the markdown file

*   **Bus 1 (Type:?                ):** `V₁ = 1.0∠0°` p.u.
*   **Bus 2 (Type:?                ):** `P₂ = 1.5` p.u., `|V₂| = 1.1` p.u.
*   **Bus 3 (Type:?                ):** `P₃ = -1.5` p.u., `Q₃ = 0.8` p.u.

Determine the bus admittance matrix **Ybus** for this system.

<img src="https://i.imgur.com/xeyCiDy.png" width=400>

**Guidance:**
*   This problem is simpler because the admittances are given directly.
*   There are no shunt elements shown other than the load itself, which we do not include in the Ybus. The Ybus represents the physical network connections, not the injections.
*   Follow the same "rules of inspection" as in Problem 1.

In [3]:


# --- Problem 2 Solution ---

# --- Given Admittances ---
y12 = 2.0 - 4.0im
y13 = 3.0 - 6.0im

# --- Step 1: Initialize Ybus ---
N = 3
Ybus_p2 = zeros(Complex{Float64}, N, N)

# --- Step 2: Populate Off-Diagonal Elements ---
# Note: There is no direct connection between bus 2 and 3
Ybus_p2[1, 2] = Ybus_p2[2, 1] = -y12
Ybus_p2[1, 3] = Ybus_p2[3, 1] = -y13
Ybus_p2[2, 3] = Ybus_p2[3, 2] = 0.0 # No connection

# --- Step 3: Populate Diagonal Elements ---
Ybus_p2[1, 1] = y12 + y13
Ybus_p2[2, 2] = y12+y23
Ybus_p2[3, 3] = y23

println("--- Final Ybus Matrix for Problem 2 ---")
show(IOContext(stdout, :limit => false), "text/plain", round.(Ybus_p2, digits=4))

--- Final Ybus Matrix for Problem 2 ---
3×3 Matrix{ComplexF64}:
  5.0-10.0im    -2.0+4.0im    -3.0+6.0im
 -2.0+4.0im   3.6667-9.0im     0.0+0.0im
 -3.0+6.0im      0.0+0.0im  1.6667-5.0im